In [3]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn import svm
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'unknown'

def title_map(title):
    if title in ['Mr']:
        return 1
    elif title in ['Master']:
        return 3
    elif title in ['Mme','Mrs','Ms', 'Mlle', 'Miss','Countess']:
        return 4
    elif title in ['Dr']:
        return 6
    return 2


#print(pd.Series.value_counts(train['Survived']))
# check for gender submission
# X = train['Sex'].astype('category').cat.codes
# X = X.reshape(-1,1)
# Y = train['Survived']
# model = GaussianNB()
# model.fit(X,Y)
# X = test['Sex'].astype('category').cat.codes
# X = X.reshape(-1,1)
# y_pred = model.predict(X)
# y_pred = y_pred.reshape(-1,1)
#train = train.dropna()

# feature engineering
data = train.append(test)
data['Family'] = data['Parch']+data['SibSp']
data['Embarked'].fillna('S', inplace = True)
data['Embarked'] = data['Embarked'].astype('category').cat.codes
# for i in range(len(data['Cabin'])):
#     if len(str(data['Cabin'].iloc[i])) > 0:
#         data['Cabin'].iloc[i] = str(data['Cabin'].iloc[i])[0]
data['Cabin']=data['Cabin'].apply(lambda x:str(x)[0])
data['Name'] = data['Name'].apply(get_title).apply(title_map)
#data['Family'][data['Family'] > 0.0 ] = 1.0
data = data.drop(['Ticket'], axis = 1)
# fill missing values in Age column and categorize ages
groups =  data.groupby('Pclass')
for key, items in groups:
    data['Age'].fillna(pd.Series.median(groups.get_group(key)['Age']), inplace=True)
    data['Fare'].fillna(pd.Series.median(groups.get_group(key)['Fare']), inplace=True)
# data['Age'][data['Age'] <= 18.0 ] = 10.0
# data['Age'][(data['Age'] > 18.0) & (data['Age'] <= 30.0)] = 25.0
# data['Age'][(data['Age'] > 30.0) & (data['Age'] <= 45.0)] = 40.0
# data['Age'][(data['Age'] > 45.0) & (data['Age'] <= 60.0)] = 50.0
# data['Age'][data['Age'] > 60.0] = 70.0
# # binning Fare
# data['Fare'][data['Fare'] <= 50.0 ] = 25.0
# data['Fare'][(data['Fare'] > 50.0) & (data['Fare'] <= 70.0)] = 60.0
# data['Fare'][(data['Fare'] > 70.0) & (data['Fare'] <= 150.0)] = 100.0
# data['Fare'][data['Fare'] > 150.0] = 150.0
data['Sex'] = data['Sex'].astype('category').cat.codes
data['Fare'] = data['Fare'].astype(int)
data['Cabin'] = data['Cabin'].astype('category').cat.codes

# training model
train = data[:len(train)]
test = data[len(train):]
test = test.drop(['Survived'], axis = 1)
X = train.drop(['Survived'], axis = 1)
train.drop(['PassengerId'], axis=1, inplace=True)
Y = train['Survived']
scores = cross_val_score(RandomForestClassifier(n_estimators=100), X, Y, cv = 8)
print(scores)
model = RandomForestClassifier(n_estimators=100)
model.fit(X, Y)
# test model
X = test
y_pred = model.predict(X)
y_pred.reshape(-1,1)
y_pred = pd.DataFrame(y_pred.astype(int))
y_pred=y_pred.set_index(test['PassengerId'])
csv = open("predictions.csv","w")
csv.write("PassengerId,Survived\n")
for i in range(len(y_pred[0])):
    csv.write(str(test['PassengerId'].iloc[i]) + ","+ str(y_pred[0].iloc[i])+"\n")
csv.close()
print(train)

C:\Users\remil\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[ 0.82142857  0.78571429  0.80357143  0.88392857  0.83928571  0.81981982
  0.81818182  0.82727273]
      Age  Cabin  Embarked  Fare  Name  Parch  Pclass  Sex  SibSp  Survived  \
0    22.0      8         2     7     1      0       3    1      1       0.0   
1    38.0      2         0    71     4      0       1    0      1       1.0   
2    26.0      8         2     7     4      0       3    0      0       1.0   
3    35.0      2         2    53     4      0       1    0      1       1.0   
4    35.0      8         2     8     1      0       3    1      0       0.0   
5    39.0      8         1     8     1      0       3    1      0       0.0   
6    54.0      4         2    51     1      0       1    1      0       0.0   
7     2.0      8         2    21     3      1       3    1      3       0.0   
8    27.0      8         2    11     4      2       3    0      0       1.0   
9    14.0      8         0    30     4      0       2    0      1       1.0   
10    4.0      6         2    16